# Системы хранения и обработки данных. Домашнее задание 3.
## Присяжнюк Артем Сергеевич

# Подготовка

In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2 as pg

In [2]:
conn = pg.connect(dbname="postgres", host="localhost", user="postgres", password="password", port=5432)
cursor = conn.cursor()

# Задание 1

## Создание таблиц

In [198]:
cursor.execute("""
               create table transaction (
                   transaction_id int4 NOT NULL PRIMARY KEY
                   ,product_id int4
                   ,customer_id int4 
                   ,transaction_date varchar
                   ,online_order varchar
                   ,order_status varchar
                   ,list_price float4
                   ,brand varchar
                   ,product_line varchar
                   ,product_class varchar
                   ,product_size varchar
                   ,standard_cost float4                 
               )
               """)

In [199]:
cursor.execute("""
               create table customer (
                   customer_id int4 NOT NULL PRIMARY KEY
                   ,first_name varchar
                   ,last_name varchar
                   ,gender varchar
                   ,dob varchar
                   ,job_title varchar
                   ,job_industry_category varchar
                   ,wealth_segment varchar
                   ,deceased_indicator varchar
                   ,owns_car varchar
                   ,property_valuation int4
                   ,address varchar
                   ,postcode varchar
                   ,state varchar
                   ,country varchar                  
               )
               """)

In [200]:
conn.commit()

## Обработка данных

In [86]:
import pandas as pd

df_transaction = pd.read_csv("data/transaction.csv", sep=';')
df_transaction.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82"
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10"
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48"


In [202]:
df_transaction.standard_cost = df_transaction.standard_cost.apply(lambda x: float(str(x).replace(',', '.')))
df_transaction.list_price = df_transaction.list_price.apply(lambda x: float(str(x).replace(',', '.')))

In [203]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    20000 non-null  int64  
 1   product_id        20000 non-null  int64  
 2   customer_id       20000 non-null  int64  
 3   transaction_date  20000 non-null  object 
 4   online_order      19640 non-null  object 
 5   order_status      20000 non-null  object 
 6   brand             19803 non-null  object 
 7   product_line      19803 non-null  object 
 8   product_class     19803 non-null  object 
 9   product_size      19803 non-null  object 
 10  list_price        20000 non-null  float64
 11  standard_cost     19803 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 1.8+ MB


In [87]:
df_customer = pd.read_csv("data/customer.csv", sep=';')
df_customer.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [205]:
df_customer['deceased_indicator'].unique(), df_customer['owns_car'].unique()

(array(['N', 'Y'], dtype=object), array(['Yes', 'No'], dtype=object))

In [206]:
df_customer['gender'].unique()

array(['F', 'Male', 'Female', 'U', 'Femal', 'M'], dtype=object)

In [207]:
df_customer['gender'].replace({'Female': 'F', 'Femal': 'F', 'Male': 'M'}, inplace=True)

In [208]:
df_customer['country'].unique(), df_customer['state'].unique()

(array(['Australia'], dtype=object),
 array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object))

In [209]:
df_customer['state'].replace({'New South Wales': 'NSW', 'Victoria': 'VIC'}, inplace=True)

In [210]:
df_customer.rename({'DOB': 'dob'}, axis=1, inplace=True)

In [211]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   dob                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
 10  address                4000 non-null   object
 11  postcode               4000 non-null   int64 
 12  state                  4000 non-null   object
 13  country                4000 non-null   object
 14  property_valuation     4000 non-null   int64 
dtypes: int64(3), object(1

In [212]:
transaction_names = [
    'transaction_id', 'transaction_date',
    'online_order', 'order_status',
    'list_price', 
    'product_id', 'customer_id',
    'brand',
    'product_line', 'product_class',
    'product_size', 'standard_cost'
    
]

customer_names = [
    'customer_id', 'first_name', 'last_name',
    'gender', 'dob',
    'job_title', 'job_industry_category',
    'wealth_segment', 'property_valuation', 'owns_car',
    'deceased_indicator',
    'address', 'country',
    'state', 'postcode'
]

In [213]:
df_transaction.shape, df_transaction.transaction_id.nunique()

((20000, 12), 20000)

In [214]:
df_customer.shape, df_customer.customer_id.nunique()

((4000, 15), 4000)

In [162]:
# nat_mask = df_customer.dob.apply(lambda x: x is pd.NaT)
# na_mask = df_customer.dob.isna()
# df_customer.loc[nat_mask | na_mask, "dob"] = pd.to_datetime(0) # because postgresql does not accept NaT

In [151]:
%pip install tqdm

  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [215]:
from tqdm import tqdm

for _, row_t in tqdm(df_transaction.iterrows()):
    cursor.execute(f"""
                insert into transaction ({','.join(transaction_names)})
                values (%s{', %s' * (len(transaction_names)-1)})
                on conflict do nothing
                """, row_t[transaction_names])

20000it [00:20, 960.03it/s] 


In [227]:
for _, row_c in tqdm(df_customer.iterrows()):
    cursor.execute(f"""
                insert into customer ({','.join(customer_names)})
                values (%s{', %s' * (len(customer_names)-1)})
                on conflict do nothing
                """, row_c[customer_names])

4000it [00:04, 813.92it/s]


In [228]:
conn.commit()

# Задание 2

### 2.1

In [9]:
cursor.execute(f"""
                select job_industry_category, count(customer_id)
                from customer
                group by job_industry_category
                order by count(customer_id) desc
                """)
print(cursor.fetchall())

[('Manufacturing', 799), ('Financial Services', 774), ('NaN', 656), ('Health', 602), ('Retail', 358), ('Property', 267), ('IT', 223), ('Entertainment', 136), ('Argiculture', 113), ('Telecommunications', 72)]


### 2.2

In [16]:
cursor.execute(f"""
                select c.job_industry_category, date_part('month', t.transaction_date::timestamp) as month, sum(t.list_price)
                from transaction as t left join customer as c on t.customer_id = c.customer_id
                group by month, c.job_industry_category
                order by month, c.job_industry_category
                """)
print(cursor.fetchall())

[('Argiculture', 1.0, 43513.812), ('Entertainment', 1.0, 64089.934), ('Financial Services', 1.0, 366383.78), ('Health', 1.0, 286860.44), ('IT', 1.0, 107783.414), ('Manufacturing', 1.0, 365232.38), ('NaN', 1.0, 316819.72), ('Property', 1.0, 100686.97), ('Retail', 1.0, 182375.72), ('Telecommunications', 1.0, 31210.2), (None, 1.0, 1102.94), ('Argiculture', 2.0, 60016.81), ('Entertainment', 2.0, 63965.99), ('Financial Services', 2.0, 375961.8), ('Health', 2.0, 268525.75), ('IT', 2.0, 93961.79), ('Manufacturing', 2.0, 389260.53), ('NaN', 2.0, 260711.89), ('Property', 2.0, 112255.41), ('Retail', 2.0, 146107.58), ('Telecommunications', 2.0, 27678.2), ('Argiculture', 3.0, 49048.492), ('Entertainment', 3.0, 77122.44), ('Financial Services', 3.0, 322268.28), ('Health', 3.0, 290575.88), ('IT', 3.0, 100212.4), ('Manufacturing', 3.0, 377784.72), ('NaN', 3.0, 278553.9), ('Property', 3.0, 125143.59), ('Retail', 3.0, 172366.56), ('Telecommunications', 3.0, 24541.277), ('Argiculture', 4.0, 45978.31), (

### 2.3

In [56]:
cursor.execute(f"""
                select t.brand, count(t.transaction_id) as counts
                from transaction as t 
                left join customer as c
                on t.customer_id = c.customer_id
                where t.online_order = 'true' and c.job_industry_category = 'IT' 
                group by t.brand
                order by counts desc
                """)
print(cursor.fetchall())

[('Solex', 102), ('Norco Bicycles', 93), ('WeareA2B', 90), ('Giant Bicycles', 89), ('Trek Bicycles', 85), ('OHM Cycles', 78), ('NaN', 8)]


### 2.4

In [5]:
cursor.execute(f"""
                select customer_id, sum(list_price) as total_sum, 
                max(list_price), min(list_price), count(*) as total_count
                from transaction
                group by customer_id
                order by total_sum desc, total_count desc
                """)
result_group_by_fp4 = cursor.fetchall()
result_group_by_fp4

[(2183, 19071.322, 2005.66, 230.91, 14),
 (1129, 18349.27, 1992.93, 290.62, 13),
 (1597, 18052.68, 2091.47, 360.4, 12),
 (941, 17898.459, 2091.47, 1057.51, 10),
 (2788, 17258.94, 2083.94, 183.86, 11),
 (936, 17160.24, 2005.66, 183.86, 12),
 (1887, 17133.932, 2091.47, 688.63, 11),
 (1302, 17035.83, 1977.36, 71.16, 13),
 (1140, 16199.24, 2083.94, 183.86, 13),
 (2309, 16122.341, 2091.47, 290.62, 12),
 (729, 15825.999, 2091.47, 586.45, 10),
 (1103, 15447.92, 1977.36, 230.91, 12),
 (1317, 15370.81, 2091.47, 569.56, 9),
 (2874, 15091.91, 2005.66, 544.05, 11),
 (2762, 15071.26, 1997.68, 183.86, 10),
 (255, 14949.91, 1810.0, 441.49, 12),
 (1672, 14908.461, 1977.36, 363.01, 13),
 (1329, 14818.119, 1793.43, 543.39, 11),
 (322, 14782.2295, 2005.66, 12.01, 12),
 (3048, 14756.089, 1890.39, 543.39, 13),
 (2860, 14673.35, 1812.75, 175.89, 10),
 (999, 14662.139, 2091.47, 495.72, 10),
 (1460, 14633.24, 2091.47, 1228.07, 8),
 (2894, 14621.73, 2083.94, 1129.13, 9),
 (3060, 14609.221, 1992.93, 533.51, 11)

In [7]:
cursor.execute(f"""
                select customer_id, sum(list_price::float8) as total_sum, 
                max(list_price), min(list_price), count(*) as total_count
                from transaction
                group by customer_id
                order by total_sum desc, total_count desc
                """)
result_group_by_fp8 = cursor.fetchall()
result_group_by_fp8

[(2183, 19071.320251464844, 2005.66, 230.91, 14),
 (1129, 18349.270141601562, 1992.93, 290.62, 13),
 (1597, 18052.67984008789, 2091.47, 360.4, 12),
 (941, 17898.4599609375, 2091.47, 1057.51, 10),
 (2788, 17258.939895629883, 2083.94, 183.86, 11),
 (936, 17160.239944458008, 2005.66, 183.86, 12),
 (1887, 17133.93017578125, 2091.47, 688.63, 11),
 (1302, 17035.82992553711, 1977.36, 71.16, 13),
 (1140, 16199.239944458008, 2083.94, 183.86, 13),
 (2309, 16122.340118408203, 2091.47, 290.62, 12),
 (729, 15825.999938964844, 2091.47, 586.45, 10),
 (1103, 15447.92007446289, 1977.36, 230.91, 12),
 (1317, 15370.809997558594, 2091.47, 569.56, 9),
 (2874, 15091.910278320312, 2005.66, 544.05, 11),
 (2762, 15071.260147094727, 1997.68, 183.86, 10),
 (255, 14949.910034179688, 1810.0, 441.49, 12),
 (1672, 14908.46011352539, 1977.36, 363.01, 13),
 (1329, 14818.119873046875, 1793.43, 543.39, 11),
 (322, 14782.23009300232, 2005.66, 12.01, 12),
 (3048, 14756.089965820312, 1890.39, 543.39, 13),
 (2860, 14673.349

In [8]:
cursor.execute(f"""
               select customer_id, total_sum, total_max, total_min, total_count
               from (
                select customer_id, 
                sum(list_price) over ( partition by customer_id ) as total_sum, 
                max(list_price) over ( partition by customer_id ) as total_max, 
                min(list_price) over ( partition by customer_id ) as total_min, 
                count(*) over ( partition by customer_id ) as total_count,
                row_number() over ( partition by customer_id ) as row_num
                from transaction
                )
                where row_num = 1
                order by total_sum desc, total_count desc
                """)
result_window_fp4 = cursor.fetchall()
result_window_fp4

[(2183, 19071.32, 2005.66, 230.91, 14),
 (1129, 18349.271, 1992.93, 290.62, 13),
 (1597, 18052.68, 2091.47, 360.4, 12),
 (941, 17898.459, 2091.47, 1057.51, 10),
 (2788, 17258.941, 2083.94, 183.86, 11),
 (936, 17160.24, 2005.66, 183.86, 12),
 (1887, 17133.93, 2091.47, 688.63, 11),
 (1302, 17035.828, 1977.36, 71.16, 13),
 (1140, 16199.24, 2083.94, 183.86, 13),
 (2309, 16122.34, 2091.47, 290.62, 12),
 (729, 15826.0, 2091.47, 586.45, 10),
 (1103, 15447.919, 1977.36, 230.91, 12),
 (1317, 15370.809, 2091.47, 569.56, 9),
 (2874, 15091.91, 2005.66, 544.05, 11),
 (2762, 15071.26, 1997.68, 183.86, 10),
 (255, 14949.91, 1810.0, 441.49, 12),
 (1672, 14908.46, 1977.36, 363.01, 13),
 (1329, 14818.119, 1793.43, 543.39, 11),
 (322, 14782.2295, 2005.66, 12.01, 12),
 (3048, 14756.089, 1890.39, 543.39, 13),
 (2860, 14673.349, 1812.75, 175.89, 10),
 (999, 14662.14, 2091.47, 495.72, 10),
 (1460, 14633.239, 2091.47, 1228.07, 8),
 (2894, 14621.729, 2083.94, 1129.13, 9),
 (3060, 14609.22, 1992.93, 533.51, 11)

In [9]:
cursor.execute(f"""
               select customer_id, total_sum, total_max, total_min, total_count
               from (
                select customer_id, 
                sum(list_price::float8) over ( partition by customer_id ) as total_sum, 
                max(list_price) over ( partition by customer_id ) as total_max, 
                min(list_price) over ( partition by customer_id ) as total_min, 
                count(*) over ( partition by customer_id ) as total_count,
                row_number() over ( partition by customer_id ) as row_num
                from transaction
                )
                where row_num = 1
                order by total_sum desc, total_count desc
                """)
result_window_fp8 = cursor.fetchall()
result_window_fp8

[(2183, 19071.320251464844, 2005.66, 230.91, 14),
 (1129, 18349.270141601562, 1992.93, 290.62, 13),
 (1597, 18052.67984008789, 2091.47, 360.4, 12),
 (941, 17898.4599609375, 2091.47, 1057.51, 10),
 (2788, 17258.939895629883, 2083.94, 183.86, 11),
 (936, 17160.239944458008, 2005.66, 183.86, 12),
 (1887, 17133.93017578125, 2091.47, 688.63, 11),
 (1302, 17035.82992553711, 1977.36, 71.16, 13),
 (1140, 16199.239944458008, 2083.94, 183.86, 13),
 (2309, 16122.340118408203, 2091.47, 290.62, 12),
 (729, 15825.999938964844, 2091.47, 586.45, 10),
 (1103, 15447.92007446289, 1977.36, 230.91, 12),
 (1317, 15370.809997558594, 2091.47, 569.56, 9),
 (2874, 15091.910278320312, 2005.66, 544.05, 11),
 (2762, 15071.260147094727, 1997.68, 183.86, 10),
 (255, 14949.910034179688, 1810.0, 441.49, 12),
 (1672, 14908.46011352539, 1977.36, 363.01, 13),
 (1329, 14818.119873046875, 1793.43, 543.39, 11),
 (322, 14782.23009300232, 2005.66, 12.01, 12),
 (3048, 14756.089965820312, 1890.39, 543.39, 13),
 (2860, 14673.349

In [11]:
assert len(result_group_by_fp4) == len(result_group_by_fp8) == len(result_window_fp4) == len(result_window_fp8)

In [85]:
cursor.execute(f"""
               select customer_id, list_price, count(*) as dcounts
               from transaction
               group by customer_id, list_price
               having count(*) > 1
                """)
print(cursor.fetchall())

[(1993, 1403.5, 3), (394, 1483.2, 2), (3280, 1762.96, 2), (1560, 60.34, 2), (1648, 1403.5, 2), (2183, 2005.66, 2), (621, 1992.93, 2), (396, 1231.15, 2), (1274, 235.63, 2), (2169, 1945.43, 2), (594, 1769.64, 2), (2465, 1458.17, 2), (3176, 742.54, 2), (3335, 1057.51, 2), (1150, 1890.39, 2), (2476, 1240.31, 2), (809, 742.54, 2), (1627, 1228.07, 2), (1101, 642.7, 2), (2069, 1992.93, 2), (2884, 175.89, 2), (3476, 230.91, 2), (3439, 792.9, 2), (280, 1765.3, 2), (611, 1775.81, 2), (1866, 416.98, 2), (999, 2091.47, 2), (1282, 688.63, 2), (545, 2091.47, 2), (739, 774.53, 2), (1912, 12.01, 2), (2686, 1793.43, 2), (2750, 1228.07, 2), (2046, 2091.47, 2), (8, 1129.13, 2), (3158, 1148.64, 2), (2637, 2091.47, 2), (1625, 2091.47, 3), (969, 1240.31, 2), (2070, 2091.47, 2), (346, 912.52, 2), (2162, 792.9, 2), (75, 1057.51, 2), (186, 1274.93, 2), (1558, 2091.47, 2), (655, 2091.47, 2), (1999, 1240.31, 2), (2697, 1636.9, 2), (3154, 1311.44, 2), (1015, 1403.5, 2), (3133, 1810.0, 2), (2700, 1073.07, 2), (288

Так как все функции аггрегационные, можно взять первую строку для каждой группы, поэтому используется условие row_num = 1. 

Единственное различие между методами кроется в возвращаемой сумме. В данных есть дубликаты по customer_id и list_price, что могло бы быть причиной различия, если бы в группировке использовалось поле list_price. 

Вероятно, ожидалось, что в результате должен получится одинаковый результат, **НО** в описание задания для поля list_price использовался тип данных float4, group by и оконные функции, видимо, приводят данные к разным типам для вычислений, потому что результаты при использовании float4 получаются разные. При этом если явно приводит list_price к float8, результат получается одинаковый! 

### 2.5

In [33]:
cursor.execute(f"""
               select min(total_sum), max(total_sum)
               from (
                   select customer_id, sum(list_price) as total_sum
                   from transaction
                   group by customer_id
                )
                """)
total_sum_min, total_sum_max = cursor.fetchone()
print(f"Min: {total_sum_min}, max: {total_sum_max}")

Min: 60.34, max: 19071.322


In [39]:
cursor.execute(f"""
               select t.customer_id, c.first_name, c.last_name, t.total_sum
               from (
                   select customer_id, sum(list_price) as total_sum
                   from transaction
                   group by customer_id
               ) as t 
               left join customer as c
               on t.customer_id = c.customer_id
               order by t.total_sum asc
                """)
print(cursor.fetchall())

[(3292, 'Hamlen', 'Slograve', 60.34), (2532, 'Milli', 'Hubbert', 71.49), (2274, 'Nada', 'Reinert', 142.98), (3189, 'Abbott', 'Knaggs', 200.7), (2423, 'Lenci', 'Symcox', 202.62), (2395, 'Ashlin', 'Kesby', 230.91), (2855, 'Kim', 'Tineman', 255.35), (872, 'Suki', 'Ivachyov', 290.62), (3305, 'Neddy', 'La Grange', 306.79), (301, 'Hildy', 'Bilbrook', 361.78), (751, 'Amie', 'Dufty', 386.47998), (1325, 'Ty', 'Belliard', 416.98), (1284, 'Alexis', 'Aronov', 418.81), (1846, 'Lorette', 'Daspar', 441.49), (2328, 'Gayler', 'Shackle', 441.49), (1810, 'Royall', 'Terris', 458.74002), (1796, 'Cayla', 'Marcus', 471.26), (2089, 'Erie', 'Ballston', 477.32), (898, 'Reyna', 'Braizier', 478.16), (2863, 'Alisander', 'Fetherstone', 544.05), (3272, 'Bar', 'Frankcom', 569.56), (3392, 'Brock', 'Ride', 574.64), (1921, 'Cybill', 'Wakes', 586.45), (3396, 'Arin', 'Nappin', 586.65), (2387, 'Roy', 'Jone', 591.31), (2870, 'Aldis', 'Chazerand', 599.88), (493, 'Tommie', 'Banbridge', 607.78), (1487, 'Kaela', 'Soppett', 617.

In [66]:
cursor.execute("""
               select t.customer_id, c.first_name, c.last_name, t.total_sum
               from (
                   select customer_id, sum(list_price) as total_sum
                   from transaction
                   group by customer_id
               ) as t 
               left join customer as c
               on t.customer_id = c.customer_id
               where t.total_sum = %s::float4 or t.total_sum = %s::float4
               order by t.total_sum asc
                """, (total_sum_min, total_sum_max))
print(cursor.fetchall())

[(3292, 'Hamlen', 'Slograve', 60.34), (2183, 'Jillie', 'Fyndon', 19071.322)]


### 2.6

In [23]:
cursor.execute(f"""
               select customer_id, first_transaction
               from (
                select customer_id,
               first_value(transaction_id) over customer_date as first_transaction,
               row_number() over customer_date as row_num
               from transaction
               window customer_date as (partition by customer_id order by transaction_date::timestamp)
               )
               where row_num = 1
                """)
print(cursor.fetchall())

[(1, 9785), (2, 2261), (3, 10302), (4, 12441), (5, 2291), (6, 7096), (7, 18369), (8, 10792), (9, 8591), (10, 5956), (11, 6004), (12, 6741), (13, 17763), (14, 16253), (15, 17528), (16, 14760), (17, 1404), (18, 7092), (19, 4664), (20, 18030), (21, 14115), (22, 17002), (23, 18219), (24, 15926), (25, 16026), (26, 7674), (27, 1275), (28, 7775), (29, 10676), (30, 3214), (31, 4321), (32, 15069), (33, 6156), (34, 9044), (35, 15032), (36, 1418), (37, 13475), (38, 14512), (39, 4659), (40, 10973), (41, 15262), (42, 4762), (43, 896), (44, 11825), (45, 7322), (46, 11710), (47, 5080), (48, 17666), (49, 7212), (50, 17709), (51, 4239), (52, 19862), (53, 8283), (54, 11374), (55, 16659), (56, 7134), (57, 11296), (58, 16885), (59, 3052), (60, 3209), (61, 19872), (62, 6633), (63, 18832), (64, 13497), (65, 6378), (66, 6202), (67, 2488), (68, 4503), (69, 13881), (70, 10118), (71, 862), (72, 14096), (73, 2316), (74, 16590), (75, 5979), (76, 12392), (77, 9422), (78, 4938), (79, 17976), (80, 6621), (81, 15983)

Можем проверить, что выбрались действительно первые (по дате) транзакции

In [25]:
cursor.execute(f"""
               select customer_id, transaction_id, transaction_date, first_transaction
               from (
                select *,
               first_value(transaction_id) over customer_date as first_transaction,
               row_number() over customer_date as row_num
               from transaction
               window customer_date as (partition by customer_id order by transaction_date::timestamp)
               )
                """)
print(cursor.fetchall())

[(1, 9785, '05.01.2017', 9785), (1, 13424, '21.02.2017', 9785), (1, 14486, '27.03.2017', 9785), (1, 18970, '29.03.2017', 9785), (1, 3765, '06.04.2017', 9785), (1, 5157, '11.05.2017', 9785), (1, 13644, '19.05.2017', 9785), (1, 15663, '04.06.2017', 9785), (1, 16423, '09.12.2017', 9785), (1, 14931, '14.12.2017', 9785), (1, 94, '23.12.2017', 9785), (2, 2261, '04.05.2017', 2261), (2, 6743, '11.06.2017', 2261), (2, 16411, '24.08.2017', 2261), (3, 10302, '23.02.2017', 10302), (3, 15188, '24.03.2017', 10302), (3, 12954, '05.04.2017', 10302), (3, 11784, '21.04.2017', 10302), (3, 16725, '26.06.2017', 10302), (3, 13285, '04.09.2017', 10302), (3, 11005, '16.09.2017', 10302), (3, 19498, '19.09.2017', 10302), (4, 12441, '03.04.2017', 12441), (4, 14648, '18.06.2017', 12441), (5, 2291, '03.03.2017', 2291), (5, 19993, '28.04.2017', 2291), (5, 2534, '03.05.2017', 2291), (5, 13058, '26.05.2017', 2291), (5, 2250, '16.08.2017', 2291), (5, 7692, '14.12.2017', 2291), (6, 7096, '28.01.2017', 7096), (6, 8204, 

### 2.7

In [89]:
cursor.execute(f"""
               select max(last_date - first_date)
               from (
               select
                date_part('day', first_value(transaction_date::timestamp) over customer_date)
                as first_date, 
                date_part('day', last_value(transaction_date::timestamp) over customer_date)
                as last_date
               from transaction
               window customer_date as (partition by customer_id order by transaction_date::timestamp)
               )
                """)
max_interval = cursor.fetchone()[0]
print(max_interval)

30.0


In [113]:
cursor.execute(f"""
               select c.customer_id, c.first_name, c.last_name, c.job_title
               from (
                    select customer_id, (last_date - first_date) as max_interval
                    from (
                        select customer_id,
                            date_part('day', first_value(transaction_date::timestamp) over customer_date)
                            as first_date, 
                            date_part('day', last_value(transaction_date::timestamp) over customer_date)
                            as last_date
                            from transaction
                            window customer_date as (partition by customer_id order by transaction_date::timestamp)
                    )
               ) as max_intervals
               left join customer as c
               on max_intervals.customer_id = c.customer_id
               where max_intervals.max_interval = %s::integer
               order by customer_id asc
                """, (max_interval, ))
print(cursor.fetchall())

[(166, 'Elston', 'Oleszczak', 'Structural Analysis Engineer'), (234, 'Mead', 'Alpes', 'Chief Design Engineer'), (1022, 'Suzann', 'Wilks', 'Assistant Manager'), (1100, 'Cord', 'Braundt', 'Engineer I'), (1412, 'Antonetta', 'Dumbrall', 'Tax Accountant'), (1707, 'Alfonso', 'Readett', 'Cost Accountant'), (1725, 'Corena', 'Postlewhite', 'Junior Executive'), (1762, 'Gaye', 'Steutly', 'Research Assistant II'), (3010, 'Tobye', 'Cock', 'NaN'), (3056, 'Catlin', 'Storm', 'Health Coach II'), (3231, 'Cazzie', 'Gravell', 'NaN')]


Проверим, что выбираются все записи с максимальным интервалом в днях (30 дней)

In [109]:
cursor.execute(f"""
               select customer_id, (last_date - first_date) as mint
               from (
               select customer_id,
                date_part('day', first_value(transaction_date::timestamp) over customer_date)
                as first_date, 
                date_part('day', last_value(transaction_date::timestamp) over customer_date)
                as last_date
               from transaction
               window customer_date as (partition by customer_id order by transaction_date::timestamp)
               )
               order by mint desc, customer_id asc
                """)
print(cursor.fetchall())

[(166, 30.0), (234, 30.0), (1022, 30.0), (1100, 30.0), (1412, 30.0), (1707, 30.0), (1725, 30.0), (1762, 30.0), (3010, 30.0), (3056, 30.0), (3231, 30.0), (311, 29.0), (467, 29.0), (824, 29.0), (824, 29.0), (1102, 29.0), (1229, 29.0), (1382, 29.0), (1413, 29.0), (1435, 29.0), (1670, 29.0), (1725, 29.0), (1736, 29.0), (1818, 29.0), (2065, 29.0), (2202, 29.0), (2488, 29.0), (2594, 29.0), (2611, 29.0), (2652, 29.0), (3056, 29.0), (3143, 29.0), (11, 28.0), (97, 28.0), (139, 28.0), (205, 28.0), (323, 28.0), (340, 28.0), (384, 28.0), (546, 28.0), (567, 28.0), (568, 28.0), (612, 28.0), (648, 28.0), (671, 28.0), (711, 28.0), (767, 28.0), (769, 28.0), (856, 28.0), (867, 28.0), (989, 28.0), (1041, 28.0), (1043, 28.0), (1102, 28.0), (1129, 28.0), (1285, 28.0), (1300, 28.0), (1461, 28.0), (1587, 28.0), (1615, 28.0), (1646, 28.0), (1691, 28.0), (1721, 28.0), (1736, 28.0), (1756, 28.0), (1762, 28.0), (1762, 28.0), (1793, 28.0), (1860, 28.0), (2065, 28.0), (2221, 28.0), (2273, 28.0), (2289, 28.0), (246

# Завершение работы

In [272]:
cursor.close()
conn.close()